In [ ]:
import cv2
from deepface import DeepFace
import speech_recognition as sr
from transformers import pipeline

def recognize_speech_from_mic():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        print("Please speak something...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        print("Recognizing...")
        text = recognizer.recognize_google(audio)
        print(f"You said: {text}")
        return text
    except sr.RequestError:
        print("Could not request results from Google Speech Recognition service.")
        return None
    except sr.UnknownValueError:
        print("Could not understand audio.")
        return None

def detect_emotion_from_text(text):
    emotion_classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

    emotions = emotion_classifier(text)
    return emotions

def detect_emotion_from_video():
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, frame = video_capture.read()

        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        try:
            analysis = DeepFace.analyze(rgb_frame, actions=['emotion'], enforce_detection=False)

            dominant_emotion = analysis[0]['dominant_emotion']

            cv2.putText(frame, dominant_emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        except Exception as e:
            print("Error in emotion analysis:", e)

        cv2.imshow('Emotion Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

def main():
    print("Choose an option:")
    print("1: Detect emotions from video")
    print("2: Detect emotions from speech")
    choice = input("Enter your choice (1/2): ")

    if choice == '1':
        detect_emotion_from_video()
    elif choice == '2':
        spoken_text = recognize_speech_from_mic()
        if spoken_text:
            emotions = detect_emotion_from_text(spoken_text)
            for emotion in emotions:
                print(f"Emotion: {emotion['label']}, Score: {emotion['score']:.4f}")
    else:
        print("Invalid choice. Please restart the program.")

if __name__ == "__main__":
    main()



Choose an option:
1: Detect emotions from video
2: Detect emotions from speech
